In [8]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import gymnasium as gym
from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy

SEED = 42

env = make_vec_env(
    "CartPole-v1", 
    n_envs=10, # Q: 这个参数有什么用？
    seed=SEED
    )

model = DQN(
    policy="MlpPolicy",
    env=env,
    learning_rate=1e-3,
    buffer_size=50000, # Q: 这个参数有什么用？
    learning_starts=1000, # Q: 这个参数有什么用？
    batch_size=64,
    gamma=0.99, # Q: 这个参数有什么用？
    train_freq=4, # Q: 这个参数有什么用？
    target_update_interval=1000, # Q: 这个参数有什么用？
    exploration_fraction=0.2, # Q: 这个参数有什么用？
    exploration_final_eps=0.02, # Q: 这个参数有什么用？
    verbose=0,
    seed=SEED,
)

In [10]:
total_timesteps = 500_000
model.learn(total_timesteps=total_timesteps)

In [ ]:
mean_reward, std_reward = evaluate_policy(model, 
                                          env, 
                                          n_eval_episodes=12, # Q: 这个参数有什么用？
                                          render=False, 
                                          deterministic=True # Q: 这个参数有什么用？
                                          )

print(f"[Eval] Mean reward over 10 episodes: {mean_reward:.1f} ± {std_reward:.1f}")

[Eval] Mean reward over 10 episodes: 9.8 ± 0.4


In [12]:
save_path = "dqn_cartpole"
model.save(save_path)
print(f"Model saved to: {save_path}.zip")

Model saved to: dqn_cartpole.zip


In [17]:
import time
from stable_baselines3 import DQN

model = DQN.load("dqn_cartpole")
env = gym.make("CartPole-v1", render_mode="human")

obs, info = env.reset(seed=42)

for _ in range(1000):
    action, _ = model.predict(obs, deterministic=True)
    # 对于非向量环境，确保是标量动作
    action = int(action)                 
    obs, reward, done, truncated, info = env.step(action)
    # 用环境推荐的渲染帧率限速
    fps = env.metadata.get("render_fps", 240)
    time.sleep(1.0 / fps)
    if done or truncated:
        obs, info = env.reset()

env.close()


KeyboardInterrupt: 